In [1]:
from configuration import *
from scipy.stats import mode
import csv
import pandas as pd
import pickle
import numpy as np

## Gênero das músicas

O mapeamento entre as músicas e seus gêneros já está pronto: foi obtido na etapa 2. Aqui, basta salvá-lo no formato solicitado (CSV), com base na sua representação binária <code>genre_ids</code>:

In [2]:
genre_ids = pickle.load(file(TRACK_GENRES_BINARY, 'rb'))

In [3]:
with open(TRACK_GENRES, 'w') as csv_file:
    writer = csv.writer(csv_file)
    writer.writerow(['track_id', 'genre_id'])
    for key, value in genre_ids.items():
       writer.writerow([key, value])

## Gênero das rádios

In [4]:
print("Usaremos {} registros.".format(n_records))

Usaremos 10000 amostras.


### Inserção dos genre_id

In [5]:
df = pd.read_csv(DETECTIONS, delimiter="|", nrows=n_records)

In [6]:
df.head()

,created_at,audio_source_id,track_id
0,2014-10-01 00:00:35 UTC,9088,690
1,2014-10-01 00:00:37 UTC,288,41082
2,2014-10-01 00:00:38 UTC,345,148509
3,2014-10-01 00:00:38 UTC,493,33022
4,2014-10-01 00:00:36 UTC,1724,4539


In [7]:
df['genre_id'] = df['track_id'].apply(lambda track_id: genre_ids[track_id])

Há várias formas de definir o gênero da rádio. A mais simples talvez seja associar a ela o gênero da música mais frequente. Em outras palavras, _o gênero musical de uma rádio é o gênero musical da música mais tocada nela_. É o que faremos a seguir. Contudo, apenas a título de exemplo, outra forma de definir o gênero de uma rádio através de uma sequência de frequências relativas das músicas, de gêneros distintos, mais tocadas. Por exemplo, poderíamos dizer que uma rádio qualquer é 80% rock clássico e 20% rock pesado, supondo que "rock clássico" e "rock pesado" fossem dois grupos (_clusters_) identificados na etapa 2.

In [8]:
doi = df[['audio_source_id', 'genre_id', 'created_at']] # Data of interest

In [9]:
df2 = doi.groupby(['audio_source_id', 'genre_id']).agg(['count'])

In [10]:
df2.head()

,,created_at
,,count
audio_source_id,genre_id,
1,0,2
3,0,7
5,0,3
6,0,13
7,0,2


In [11]:
df3=df2['created_at']['count']
df3=df3.reset_index()

In [12]:
df3.head()

,audio_source_id,genre_id,count
0,1,0,2
1,3,0,7
2,5,0,3
3,6,0,13
4,7,0,2


In [13]:
__source_ids = []
__genre_ids = []

for name, group in df3.groupby('audio_source_id'):
    idx = group['count'].idxmax()
    
    __source_ids.append(name)
    __genre_ids.append(df3.iloc[idx]['genre_id'])

In [14]:
output = pd.DataFrame({'audio_source_id': __source_ids, 'genre_id': __genre_ids})

In [15]:
output.to_csv(SOURCE_GENRES, index=False)